In [1]:
localfileName = "f-17"

In [2]:
import ee
import sys
sys.path.insert(1, "../apGIS/")
import apgis
from apgis.apgeojson import GeoJSON
from apgis.apfield import Field
import apgis.geespatial as spatial
from apgis.apdate import Date
import apgis.apexception as apexception 

In [3]:
# apgis.eeInitialize()
import geemap
Map = geemap.Map()
Map.add_basemap("SATELLITE")

In [4]:
def genSZRender(ndviLayerCol, ndvi: ee.Image, field: Field) -> ee.FeatureCollection:
    try:
        def applySNIC(clusterImage):

            seeds = ee.Algorithms.Image.Segmentation.seedGrid(size=round(threshPixel*2), gridType='hex')
            geoClusterImage = (ee.Algorithms.Image.Segmentation.SNIC(image=clusterImage, size=round(threshPixel*5),
                                                                     compactness=5, seeds=seeds).select(['clusters'])
                               .clip(clusterImage.geometry()))

            return geoClusterImage

        cleanSZ = spatial.genCleanSZ(ndviLayerCol, field)       
        
        if cleanSZ.size().getInfo() == 0:
            raise apexception.EEEmptyCollectionError

        areaSZ = cleanSZ.map(spatial.setArea)
        
#         area = spatial.getArea(ee.Geometry(field.eeAOI, "HA"))
#         if 0.5 < area < 1:
#             divisor =

        threshArea = round(field.eeAOI.area(5).multiply(0.2).getInfo())
        threshPixel = round(threshArea / 100)

        bigSZLayers, smallSZLayers = spatial.splitSZLayers(areaSZ, threshArea)

        if bigSZLayers.size().getInfo() != 0: 
            clusterImageList = spatial.areaLayers(bigSZLayers)
            clusterCollection = ee.ImageCollection.fromImages(clusterImageList)

            geoClusters = clusterCollection.map(applySNIC)

            clusterFeatureList = ee.List(geoClusters.iterate(spatial.getClusterFeatureList, ee.List([]))).flatten()
            SNICCol = ee.FeatureCollection(clusterFeatureList).merge(smallSZLayers)    

        else:
            SNICCol = smallSZLayers

        SNICCol = ndvi.reduceRegions(collection=SNICCol, reducer=ee.Reducer.mean()).map(spatial.fixSNIC)
        SNICCol = SNICCol.map(spatial.setArea)      
        
        
        if SNICCol.size().getInfo() == 0:
            raise apexception.EEEmptyCollectionError
    
        return SNICCol

    except Exception as e:
        raise apexception.EERuntimeError(f"StressZone SNIC Render Generation Failed @ Runtime: {e}")

In [5]:
ndvi2Vis = {
    'min': 0, 
    'max': 10, 
    'palette': ['FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901', '66A000', '529400', 
                '3E8601', '207401', '056201', '004C00', '023B01', '012E01', '011D01', '011301']
}

S2TC = {
    'min': 0, 
    'max': 255,
    'bands': ['TCI_R', 'TCI_G', 'TCI_B']
}

In [6]:
drange = (Date("2019-01-20"), Date("2020-09-30"))
field = Field(localfile=f"{localfileName}.geojson")

S2 = apgis.gee.genCollection(sensor="L2A", daterange=drange, geometry=field.eeROI)\
.filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 15))

print(S2.size().getInfo())

32


In [7]:
print(spatial.getArea(ee.Geometry(field.geometry), "HA"))

1.816


In [8]:
image = S2.first().clip(field.eeROI)

In [9]:
Map.addLayer(field.eeAOI, {}, "bounds")
Map.addLayer(image, S2TC, "TC")
Map.centerObject(field.eeROI,zoom=15)

In [10]:
Map

Map(center=[11.317747000638239, 79.70686199982545], controls=(WidgetControl(options=['position'], widget=HBox(…

In [11]:
ndvi = apgis.index.generateIndex(image=image, sensor="L2A", index="NDVI").clip(field.eeAOI)
# ndmi = apgis.index.generateIndex(image=image, sensor="L2A", index="NDMI").clip(spatial.genBufferGeo(field, 50))

In [12]:
ndviRender = spatial.renderNDVI(ndvi)
# ndmiRender = spatial.renderNDMI(ndmi)

ndviLayerRender = spatial.layerCoding(ndviRender)
# ndmiLayerRender = spatial.layerCoding(ndmiRender)

ndviLayerCol = spatial.mergeLayerPolygons(ndviLayerRender)
# ndmiLayerCol = spatial.mergeLayerPolygons(ndmiLayerRender)

# indexes = ndvi.addBands(ndmi)

# rawData = spatial.accumulateRawValue(indexes, field)
# scoredData = spatial.scoreBuilder(rawData)

In [13]:
SNICCol = genSZRender(ndviLayerCol, ndvi, field)

In [14]:
Map.addLayer(ndvi, ndvi2Vis, "ndvi")

In [15]:
SNICCol.size().getInfo()

5

In [16]:
Map.addLayer(SNICCol, {}, "sz")

In [17]:
# marginal1 = ["f-31", "f-32"]

In [18]:
# marginal2 = ["f-23","f-33","f-34"]

In [19]:
# small = ["f-25","f-19","f-17","f-41","f-28","f-12"]

In [20]:
# semimedium1=  ["f-15","f-04","f-37","f-06","f-10","f-09"] #"f-26","f-29","f-38","f-21","f-01","f-14","f-08","f-02", "f-16"]

In [21]:
# semimedium2 = ["f-30","f-20","f-27","f-22","f-39","f-40"]

In [22]:
# medium1 = ["f-35", "f-03", "f-07"]

In [23]:
# medium2 = ["f-24","f-05","f-53", "f-36", "f-18"]

In [24]:
# medium3 = ["f-13", "f-11"]

In [25]:
# large1 = ["f-43","f-52","f-51","f-56","f-54","f-42","f-48"]

In [26]:
# large2 = ["f-44","f-47","f-45","f-50","f-46"]

In [27]:
# filenames = large2
# print(filenames)

In [28]:
# Map

In [29]:
# for index, file in enumerate(filenames):
#     drange = (Date("2019-06-30"), Date("2020-09-30"))
#     field = Field(localfile=f"{file}.geojson")

#     S2 = apgis.gee.genCollection(sensor="L2A", daterange=drange, geometry=field.eeROI)\
#     .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 15))
#     print(index+1)
    
#     image = S2.first().clip(field.eeROI)
          
#     ndvi = apgis.index.generateIndex(image=image, sensor="L2A", index="NDVI").clip(field.eeAOI)
# #     ndmi = apgis.index.generateIndex(image=image, sensor="L2A", index="NDMI").clip(spatial.genBufferGeo(field, 50))
          
#     ndviRender = spatial.renderNDVI(ndvi)
# #     ndmiRender = spatial.renderNDMI(ndmi)
    
#     Map.centerObject(field.eeROI,zoom=15)
#     Map.addLayer(field.eeAOI, {}, f"bounds-{index}")
    
#     print("Collection size",S2.size().getInfo())
    
#     ndviLayerRender = spatial.layerCoding(ndviRender)
# #     ndmiLayerRender = spatial.layerCoding(ndmiRender)
    
#     ndviLayerCol = spatial.mergeLayerPolygons(ndviLayerRender)
# #     ndmiLayerCol = spatial.mergeLayerPolygons(ndmiLayerRender)
    
#     print(spatial.getArea(ee.Geometry(field.geometry), "HA"))
    
# #     indexes = ndvi.addBands(ndmi)

# #     rawData = spatial.accumulateRawValue(indexes, field)
# #     scoredData = spatial.scoreBuilder(rawData)
#     SNICCol = genSZRender(ndviLayerCol, ndvi, field)
          
#     print("Stress zones:", SNICCol.size().getInfo())
    
#     Map.addLayer(SNICCol, {}, f"SZ-{index}")